# One-way Repeated Measures ANOVA: A further example

In this notebook I provide a further example of a repeated measures design ANOVA. The data used is simulated and hypothetically relates to the reported pain level experienced by patients at three different time points as treatment progresses for some type of injury. Scores are self-report measures on a scale of 0 (least pain) to 60 (extremely painful). 

In this analysis I will conduct Mauchly's test of sphericity, conduct an ANOVA using the statsmodel AnovaRM method, and follow this up with appropriate post-hoc tests should the ANOVA find a signficant difference between the three conditions of the independent variable. 

In [1]:
# Importing the software libraries. 

import numpy as np
import pandas as pd
from statsmodels.stats.anova import AnovaRM
import pingouin as pg

In [2]:
# Importing the dataset for analysis. Note: the data is already organised in the long and thin format so no modification
# needs to be made to the variables to make it easier to run the analysis. 

pain_df = pd.read_csv('pain_therapy_long.csv')

pain_df.head()

,participant,time,pain
0,1,time_1,51
1,2,time_1,50
2,3,time_1,42
3,4,time_1,42
4,5,time_1,43


### Mauchly's test of sphericity

In [3]:
# Running Mauchly's test using the pg.sphericity method.

pg.sphericity(data = pain_df, dv = 'pain', subject = 'participant', within = 'time')

SpherResults(spher=True, W=0.9160459172485866, chi2=1.1399542383285237, dof=2, pval=0.5655383785422423)

We can see from the above result that Mauchly's test is non-significant. Mauchly's: W = 0.92, X2(2) = 1.14, p = 0.57
This suggests we are probably safe to assume sphericity in the data and do not necessarily need to conduct a robust ANOVA test with an adjustment to the p-value. 

### Running the repeated measures ANOVA

In [4]:
# Running the RM Anova analysis. First specifying the parameters for the model, then fitting the model, and printing the result.

aovrm = AnovaRM(pain_df, 'pain', 'participant', within = ['time'])
res = aovrm.fit()

print(res)

              Anova
     F Value Num DF  Den DF Pr > F
----------------------------------
time 38.8674 2.0000 28.0000 0.0000



The above ANOVA indicates that we have a statistically signficant difference between the three time points in terms of the level of pain reported by participants. As a consequence we now need to follow this result up with post-hoc tests that compare all pairings of levels of the independent variable (time). In this case I will use Bonferroni corrected pairwise comparisons to control for the familywise error rate (increased likelihood of a Type 1 error). 

To conduct the post-hoc tests I will use the pairwise_tests method from the pingouin software library. Before doing so I will first obtain a descriptive statistics for each level of the indepenent variable separately. This will be useful to have to hand as it will aid in the interpretation of the results of the post-hoc tests. 

### Obtaining mean scores for each level of the time (IV) variable

In [5]:
# Creating a new object for each of the levels of the independent variable (time):

time_1 = pain_df.loc[pain_df.time == 'time_1']
time_1

,participant,time,pain
0,1,time_1,51
1,2,time_1,50
2,3,time_1,42
3,4,time_1,42
4,5,time_1,43
5,6,time_1,46
6,7,time_1,46
7,8,time_1,54
8,9,time_1,47
9,10,time_1,50


In [6]:
time_2 = pain_df.loc[pain_df.time == 'time_2']
time_2

,participant,time,pain
15,1,time_2,31
16,2,time_2,38
17,3,time_2,38
18,4,time_2,39
19,5,time_2,32
20,6,time_2,29
21,7,time_2,27
22,8,time_2,38
23,9,time_2,35
24,10,time_2,33


In [7]:
time_3 = pain_df.loc[pain_df.time == 'time_3']
time_3

,participant,time,pain
30,1,time_3,42
31,2,time_3,33
32,3,time_3,34
33,4,time_3,42
34,5,time_3,42
35,6,time_3,30
36,7,time_3,36
37,8,time_3,30
38,9,time_3,38
39,10,time_3,40


In [8]:
# Obtaining mean scores for each of the three levels saved in the above objects.
time_1.pain.describe()

count    15.000000
mean     47.400000
std       3.775863
min      42.000000
25%      44.500000
50%      47.000000
75%      50.000000
max      54.000000
Name: pain, dtype: float64

In [9]:
time_2.pain.describe()

count    15.000000
mean     34.133333
std       3.700708
min      27.000000
25%      31.500000
50%      35.000000
75%      37.500000
max      39.000000
Name: pain, dtype: float64

In [11]:
time_3.pain.describe()

count    15.000000
mean     38.066667
std       4.712698
min      30.000000
25%      35.000000
50%      40.000000
75%      42.000000
max      45.000000
Name: pain, dtype: float64

In [12]:
print(f"Time 1 mean: {time_1.pain.mean():.2f}")
print(f"Time 2 mean: {time_2.pain.mean():.2f}")
print(f"Time 3 mean: {time_3.pain.mean():.2f}")

Time 1 mean: 47.40
Time 2 mean: 34.13
Time 3 mean: 38.07


Above we can see an eight number summary of the pain rating at each time point and The mean for each group printed for convenience. 

### Post-hoc tests using the Bonferroni correction

In [13]:
# As we have a significant ANOVA result we would now want to perform post-hoc comparisons on all pairs of conditions 
# of the independent variable (time). We can do this using pingouin's pairwise_tests method. Below I have used Bonferroni
# corrected pairwise comparisons.

post_res = pg.pairwise_tests(data = pain_df, dv = 'pain', within = 'time', subject = 'participant', 
                            parametric = True, padjust = 'bonf', effsize = 'hedges')

post_res

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,time,time_1,time_2,True,True,10.176019,14.0,two-sided,7.522482e-08,2.256745e-07,bonf,1.829e+05,3.452774
1,time,time_1,time_3,True,True,5.590425,14.0,two-sided,6.658695e-05,1.997608e-04,bonf,409.965,2.126692
2,time,time_2,time_3,True,True,-2.402381,14.0,two-sided,3.072845e-02,9.218536e-02,bonf,2.249,-0.903234


From the above pairwise comparisons we can see that mean pain scores differed significantly between time 1 and time 2, and between time 1 and time 3, but did not differ signficantly between time 2 and time 3. The results of each t-test are reported below:

- Time 1 v Time 2: t(14) = 10.18, p < 0.001. Participants experienced signficantly more pain at Time 1 (Mean = 47.40) than at Time 2 (Mean = 34.13). 
- Time 1 v Time 3: t(14) = 5.59, p < 0.001. Participants experienced signficantly more pain at Time 1 (Mean = 47.40) than at Time 3 (Mean = 38.07). 
- Time 2 v Time 3: t(14) = 2.40, p = 0.092. Participants did not experience a signficant difference in pain between Time 2 (Mean = 34.13) than at Time 3 (Mean = 38.07).